weights and biases

In [0]:
!pip install --upgrade wandb

     |████████████████████████████████| 1.3MB 2.8MB/s 
     |████████████████████████████████| 92kB 29.1MB/s 
     |████████████████████████████████| 460kB 21.5MB/s 
     |████████████████████████████████| 92kB 27.9MB/s 
     |████████████████████████████████| 102kB 29.6MB/s 
     |████████████████████████████████| 235kB 37.2MB/s 
     |████████████████████████████████| 71kB 29.3MB/s 
     |████████████████████████████████| 256kB 45.2MB/s 
     |████████████████████████████████| 184kB 49.5MB/s 
  Created wheel for watchdog: filename=watchdog-0.9.0-cp36-none-any.whl size=73652 sha256=9490ca24a156f7cf9830dbf19a63cc9740ded3fef9958ed5683f6f46b8e0aab7
  Stored in directory: /root/.cache/pip/wheels/61/1d/d0/04cfe495619be2095eb8d89a31c42adb4e42b76495bc8f784c
  Created wheel for gql: filename=gql-0.1.0-cp36-none-any.whl size=5541 sha256=b8b9ac0d17e84dcacab2f3af134346361d5ee068dddb97ecf9aac29520ad52ac
  Stored in directory: /root/.cache/pip/wheels/a7/8d/65/a3247f500d675d80a01e4d2f0ee44fe99f1fae

In [0]:
!wandb login d86eadd78a23205e437360af084c318aafb64dc8

In [0]:
import wandb
run = wandb.init(project="recycle")


In [0]:
#importing the drive
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
config = run.config

In [0]:
!cp -r "/content/drive/My Drive/data" "dataset"

In [0]:
!unzip /content/dataset/dataset-original.zip 

Archive:  /content/dataset/dataset-original.zip
   creating: dataset-original/
  inflating: dataset-original/.DS_Store  
   creating: dataset-original/cardboard/
  inflating: dataset-original/cardboard/.DS_Store  
  inflating: dataset-original/cardboard/cardboard1.jpg  
   creating: __MACOSX/
   creating: __MACOSX/dataset-original/
   creating: __MACOSX/dataset-original/cardboard/
  inflating: __MACOSX/dataset-original/cardboard/._cardboard1.jpg  
  inflating: dataset-original/cardboard/cardboard10.jpg  
  inflating: __MACOSX/dataset-original/cardboard/._cardboard10.jpg  
  inflating: dataset-original/cardboard/cardboard100.jpg  
  inflating: __MACOSX/dataset-original/cardboard/._cardboard100.jpg  
  inflating: dataset-original/cardboard/cardboard101.jpg  
  inflating: __MACOSX/dataset-original/cardboard/._cardboard101.jpg  
  inflating: dataset-original/cardboard/cardboard102.jpg  
  inflating: __MACOSX/dataset-original/cardboard/._cardboard102.jpg  
  inflating: dataset-original/card

In [0]:
#importing the required packages
from fastai.vision import *
from fastai.metrics import error_rate
from wandb.fastai import WandbCallback

In [0]:
#creating directory
path = Path('/content/dataset-original')
# dest = path/'black'
# dest.mkdir(parents = True, exist_ok = True)

# dest = path/'teddy'
# dest.mkdir(parents=True, exist_ok=True)

# dest = path/'grizzly'
# dest.mkdir(parents=True, exist_ok=True)


In [0]:
path.resolve()

PosixPath('/content/dataset-original')

In [0]:
#view data
data = ImageDataBunch.from_folder(path, train='.', valid_pct = 0.2, seed=66,
                                  ds_tfms = get_transforms(), size = 224).normalize(imagenet_stats)
data.show_batch(rows = 5, figsize = (10,10))

In [0]:
print(data.classes)
print(len(data.train_ds))
print(len(data.valid_ds))

['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
2022
505


#Training

In [0]:
learn = cnn_learner(data, models.resnet50, metrics = error_rate,callback_fns=WandbCallback)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 110MB/s]


In [0]:
learn.fit_one_cycle(50)

epoch,train_loss,valid_loss,error_rate,time


In [0]:
learn.recorder.plot_losses()

In [0]:
learn.save('/content/drive/My Drive/stage-1final')
learn.export('/content/drive/My Drive/exportalgo.pkl')

In [0]:
learn.unfreeze()

In [0]:
learn.lr_find()
learn.recorder.plot()

In [0]:
learn.fit_one_cycle(3,max_lr=slice(3e-5,3e-4))

This doesn't help much so we use the stage-1 training.

In [0]:
learn.save('/content/drive/My Drive/stage-2')

#Interpretation

In [0]:
learn.load('/content/drive/My Drive/stage-1');

In [0]:
interp = ClassificationInterpretation.from_learner(learn)

In [0]:
interp.plot_confusion_matrix()

In [0]:

interp.plot_top_losses(k = 4, heatmap=False)

#Prediction

In [0]:
defaults.device = torch.device('cpu')

In [0]:
learn.export()

In [0]:
learn = load_learner(path)